In [1]:
import numpy as np
import torch

import tsvar

In [46]:
# Define random parameters
dim = 2  # Dimensionality of the process
max_jumps=50e3 * dim  # Number of events

mu = torch.tensor([0.3, 0.1])
beta = torch.tensor([
    [1.0, 0.2],
    [0.5, 0.1]
])
# Use the same constraint as GrangerBusca to allow fair comparison
alpha = torch.tensor([
    [0.7, 0.3],
    [0.0, 1.0]
])

param_dict = {
    'baseline': mu.tolist(),
    'beta': beta.tolist(),
    'adjacency': alpha.tolist(),
}

In [48]:
from experiments_utils import generate_parameters

dim = 10
max_jumps=20e3 * dim

param_dict = generate_parameters(dim=dim)

mu = torch.tensor(param_dict['baseline'])
beta = torch.tensor(param_dict['beta'])
alpha = torch.tensor(param_dict['adjacency'])

---

In [49]:
coeffs_true = torch.cat((mu, beta.flatten(), alpha.flatten())).numpy()
print('  - Simulate lots of data...')
# Simulate lots of data
wold_sim = tsvar.simulate.MultivariateWoldSimulator(
    mu_a=mu, alpha_ba=alpha, beta_ba=beta)
events = wold_sim.simulate(max_jumps=max_jumps, seed=None)
events = [torch.tensor(ev, dtype=torch.float) for ev in events]
end_time = max(map(max, events))
print((f"    - Simulated {sum(map(len, events)):,d} events "
       f"with end time: {end_time}"))
print("    - Events:")
for i, events_i in enumerate(events):
    print(f"      - dim {i:>2d} ({len(events_i):>5d} events):", events_i.numpy())

  - Simulate lots of data...
    - Simulated 200,000 events with end time: 129328.7109375
    - Events:
      - dim  0 (15641 events): [      1.43       8.66      56.05 ...  129320.01  129320.09  129324.05]
      - dim  1 (26173 events): [     14.10      26.37      42.33 ...  129313.47  129317.32  129318.38]
      - dim  2 (22957 events): [     44.90      45.68      53.20 ...  129323.62  129326.46  129326.78]
      - dim  3 (28957 events): [     27.57      53.14      54.86 ...  129314.80  129325.15  129328.71]
      - dim  4 (14942 events): [      7.24      42.18      63.09 ...  129312.20  129314.23  129316.34]
      - dim  5 (17449 events): [     22.73      75.95      82.20 ...  129310.24  129322.94  129326.55]
      - dim  6 (16937 events): [     77.06      80.25     103.82 ...  129317.19  129318.50  129319.31]
      - dim  7 (12699 events): [    615.59     657.96     658.23 ...  129296.70  129299.62  129306.08]
      - dim  8 (19677 events): [     27.84      29.61      39.34 ...  12

---

In [63]:
# coeffs_true_dict = param_dict
seed = 1234

MLE_N_ITER = 75000

PRINT_EVERY = 50
CALLBACK_END = '\n'


# set random seed
np.random.seed(seed)
torch.manual_seed(seed)

# Set initial guess
coeffs_start = torch.tensor(np.hstack((
    np.random.uniform(0.0, 1.0, size=dim),     # baseline
    np.random.uniform(0.0, 1.0, size=dim**2),  # beta
    np.random.uniform(0.0, 1.0, size=dim**2)   # adjacency
)))

# Extract ground truth
coeffs_true = np.hstack((coeffs_true_dict['baseline'],
                         np.array(coeffs_true_dict['beta']).flatten(),
                         np.array(coeffs_true_dict['adjacency']).flatten()))

# Define model
model = tsvar.models.WoldModelMLE(verbose=True)
model.observe(events, end_time)

# Set callback
callback = tsvar.utils.callbacks.LearnerCallbackMLE(
    coeffs_start, print_every=PRINT_EVERY, coeffs_true=coeffs_true,
    acc_thresh=0.05, dim=dim, default_end=CALLBACK_END)

# Fit model
conv = model.fit(x0=coeffs_start, optimizer=torch.optim.Adam, lr=0.5,
                 lr_sched=0.9999, tol=1e-5, max_iter=1000,
                 penalty=tsvar.priors.GaussianPrior, C=1e4,
                 seed=None, callback=callback)
coeffs_hat = model.coeffs.detach().numpy()

# Print results
print('\nConverged?', conv)
max_diff = np.max(np.abs(coeffs_true - coeffs_hat))
print(f'  - coeffs_hat:  {coeffs_hat.round(2)}')
print(f'  - coeffs_true: {coeffs_true.round(2)}')
print(f'  - max_diff: {max_diff:.4f}')

iter:    50 | dx: +1.2999e-01 | loss: 2.2196e+01 | dloss: +1.75e-02 | acc: 0.53 | f1-score: 0.59 | relerr: 21.47 | time/it: 1.23e-01     
iter:   100 | dx: +2.2845e-02 | loss: 2.2325e+01 | dloss: -1.17e-03 | acc: 0.53 | f1-score: 0.59 | relerr: 21.42 | time/it: 1.05e-01     
iter:   150 | dx: +3.0869e-02 | loss: 2.2252e+01 | dloss: -1.68e-03 | acc: 0.53 | f1-score: 0.59 | relerr: 21.08 | time/it: 1.08e-01     
iter:   200 | dx: +4.7185e-02 | loss: 2.2156e+01 | dloss: -2.16e-03 | acc: 0.53 | f1-score: 0.59 | relerr: 20.65 | time/it: 1.05e-01     
iter:   250 | dx: +9.6080e-02 | loss: 2.2031e+01 | dloss: -2.94e-03 | acc: 0.53 | f1-score: 0.59 | relerr: 20.13 | time/it: 1.05e-01     
iter:   300 | dx: +8.9695e-02 | loss: 2.1889e+01 | dloss: -2.92e-03 | acc: 0.54 | f1-score: 0.60 | relerr: 19.62 | time/it: 1.06e-01     
iter:   350 | dx: +1.3149e-01 | loss: 2.1722e+01 | dloss: -3.58e-03 | acc: 0.55 | f1-score: 0.60 | relerr: 19.11 | time/it: 1.05e-01     
iter:   400 | dx: +1.9148e-01 | lo

In [64]:
np.reshape(coeffs_hat[dim:dim+dim**2], (dim, dim))

array([[  0.00,  15.74,   0.00,  15.59,  15.71,   0.00,   0.00,  15.73,  15.91,  15.86],
       [  0.00,  16.83,   0.00,  16.90,   0.00,   0.00,   0.00,  17.14,  17.07,  16.88],
       [  0.00,  16.65,   0.00,  16.58,   0.00,   0.00,   0.00,  16.82,  16.72,  16.71],
       [  0.00,  16.99,   0.00,  17.19,   0.00,   0.00,   0.00,  16.98,  17.07,  17.04],
       [  0.00,  15.56,   0.00,  15.50,   0.00,   0.00,   0.00,  15.82,  15.46,  15.48],
       [  0.00,  16.05,   0.00,  15.95,   0.00,   0.00,   0.00,  15.80,  16.04,  15.96],
       [  0.00,  15.99,   0.00,  15.91,   0.00,   0.00,   0.00,  15.79,  16.11,  16.10],
       [  0.00,  15.20,   0.00,  14.95,   0.00,   0.00,   0.00,  15.55,  14.98,  15.13],
       [  0.00,  16.14,   0.00,  16.20,   0.00,   0.00,   0.00,  16.49,  16.55,  16.41],
       [  0.00,  16.76,   0.00,  16.76,  16.67,   0.00,   0.00,  17.03,  16.95,  16.72]])